In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

secret_key = os.getenv('SQL_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [15]:
''' OLD:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///../afl.db", sample_rows_in_table_info=10)
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, top_k = 20)
'''
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///../afl.db", sample_rows_in_table_info=10)

from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [16]:
agent_executor.invoke(
    "In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?"
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_query` with `SELECT COUNT(*) AS games_count FROM player_game_stats WHERE player_name = 'Christian Petracca' AND disposals > (SELECT disposals FROM player_game_stats WHERE player_name = 'Clayton Oliver' AND year = 2022) AND year = 2022;`


Error: (sqlite3.OperationalError) no such table: player_game_stats
[SQL: SELECT COUNT(*) AS games_count FROM player_game_stats WHERE player_name = 'Christian Petracca' AND disposals > (SELECT disposals FROM player_game_stats WHERE player_name = 'Clayton Oliver' AND year = 2022) AND year = 2022;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_schema` with `{'table_names': 'game, player_stats'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete IN

{'input': 'In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?',
 'output': 'In 2022, Christian Petracca had more disposals than Clayton Oliver in 5 games.'}

In [6]:
TEMPLATE = """You are Anthony (Huddo) Hudson, an AFL commentator and stat expert. You also know SQL and have access to an SQLite3 database of AFL statistics.
Given an input question, first create a syntactically correct SQLite3 query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

player_stats and game.

Few shot examples:
Question: "How many goals did Zak Butters kick in 2023?"
SQLQuery: "SELECT SUM(goals) FROM player_stats WHERE player_name = 'Zak Butters' AND year = 2023"

Question: "How many goals did Max Gawn kick last year?"
SQLQuery = "SELECT goals FROM player_stats WHERE player_name = 'Max Gawn' AND year = strftime('%Y', date('now', '-1 year'))"

Whenever you are asked a question, please return additional information that might be implied by the question, such as what year or round a statistic is from.

If you get no result from the query, return "No result", and suggest a new query to run.

Question: {user_question}"""

In [17]:
user_question = "In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?"
agent_executor.invoke(user_question)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'player_stats'}`



CREATE TABLE player_stats (
	round INTEGER, 
	year INTEGER, 
	player_id INTEGER, 
	player_name VARCHAR(255), 
	team VARCHAR(50), 
	opponent VARCHAR(50), 
	kicks INTEGER, 
	marks INTEGER, 
	hand_balls INTEGER, 
	disp INTEGER, 
	goals INTEGER, 
	behinds INTEGER, 
	hit_outs INTEGER, 
	tackles INTEGER, 
	rebounds INTEGER, 
	inside_50 INTEGER, 
	clearances INTEGER, 
	clangers INTEGER, 
	frees_for INTEGER, 
	frees_against INTEGER, 
	brownlow INTEGER, 
	contested_possessions INTEGER, 
	uncontested_possessions INTEGER, 
	contested_marks INTEGER, 
	marks_inside_50 INTEGER, 
	one_percenters INTEGER, 
	bounces INTEGER, 
	goal_assists INTEGER, 
	percent_time_played INTEGER, 
	game_id INTEGER, 
	PRIMARY KEY (round, year, player_id), 
	FOREIGN KEY(game_id) REFERENCES game (game_id)
)

/*
10 rows from player_stats table:
round	year	pl

{'input': 'In how many games did Christian Petracca have more disposals than Clayton Oliver in 2022?',
 'output': 'In 2022, Christian Petracca had more disposals than Clayton Oliver in 5 games.'}

In [9]:
user_question = "How many times has Harris Andrews had more One Percenters than anyone else in that game?"
agent_executor.invoke(user_question)
# SELECT COUNT(*) FROM player_stats WHERE one_percenters = (SELECT MAX(one_percenters) FROM player_stats WHERE player_name = 'Harris Andrews');
# SELECT COUNT(*) FROM player_stats WHERE player_name = 'Harris Andrews' AND one_percenters = (SELECT MAX(one_percenters) FROM player_stats);
# BOTH WRONG. I THINK IT NEEDS TO BE GIVEN A MATCH_ID IN THE DATABASE.




> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'game'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete INTEGER, 
	hbehinds INTEGER, 
	ascore INTEGER, 
	roundname TEXT, 
	year INTEGER, 
	is_final INTEGER, 
	hscore INTEGER, 
	abehinds INTEGER, 
	is_grand_final INTEGER, 
	updated TEXT, 
	hgoals INTEGER, 
	timestr TEXT, 
	hteam TEXT, 
	winnerteamid REAL, 
	ateamid INTEGER, 
	PRIMARY KEY (game_id)
)

/*
10 rows from game table:
round	agoals	game_id	unixtime	date	hteamid	ateam	tz	venue	localtime	winner	complete	hbehinds	ascore	roundname	year	is_final	hscore	abehinds	is_grand_final	updated	hgoals	timestr	hteam	winnerteamid	ateamid
1	20	1	1490257200	2017-03-23 19:20:00	3	Richmond	+11:00	M.C.G.	2017-03-23 19:20:00	Richmond	100	5	132	

{'input': 'How many times has Harris Andrews had more One Percenters than anyone else in that game?',
 'output': 'Harris Andrews has not had more One Percenters than anyone else in any game in the database.'}

In [10]:
user_question = "What is the lowest number of disposals for which a player has received 3 brownlow votes?"
agent_executor.invoke(user_question)
# Correct!



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'game'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete INTEGER, 
	hbehinds INTEGER, 
	ascore INTEGER, 
	roundname TEXT, 
	year INTEGER, 
	is_final INTEGER, 
	hscore INTEGER, 
	abehinds INTEGER, 
	is_grand_final INTEGER, 
	updated TEXT, 
	hgoals INTEGER, 
	timestr TEXT, 
	hteam TEXT, 
	winnerteamid REAL, 
	ateamid INTEGER, 
	PRIMARY KEY (game_id)
)

/*
10 rows from game table:
round	agoals	game_id	unixtime	date	hteamid	ateam	tz	venue	localtime	winner	complete	hbehinds	ascore	roundname	year	is_final	hscore	abehinds	is_grand_final	updated	hgoals	timestr	hteam	winnerteamid	ateamid
1	20	1	1490257200	2017-03-23 19:20:00	3	Richmond	+11:00	M.C.G.	2017-03-23 19:20:00	Richmond	100	5	132	

{'input': 'What is the lowest number of disposals for which a player has received 3 brownlow votes?',
 'output': 'The lowest number of disposals for which a player has received 3 Brownlow votes is 7.'}

In [11]:
user_question = "How many games have Christian Petracca and Clayton Oliver played together?"
agent_executor.invoke(user_question)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'game'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete INTEGER, 
	hbehinds INTEGER, 
	ascore INTEGER, 
	roundname TEXT, 
	year INTEGER, 
	is_final INTEGER, 
	hscore INTEGER, 
	abehinds INTEGER, 
	is_grand_final INTEGER, 
	updated TEXT, 
	hgoals INTEGER, 
	timestr TEXT, 
	hteam TEXT, 
	winnerteamid REAL, 
	ateamid INTEGER, 
	PRIMARY KEY (game_id)
)

/*
10 rows from game table:
round	agoals	game_id	unixtime	date	hteamid	ateam	tz	venue	localtime	winner	complete	hbehinds	ascore	roundname	year	is_final	hscore	abehinds	is_grand_final	updated	hgoals	timestr	hteam	winnerteamid	ateamid
1	20	1	1490257200	2017-03-23 19:20:00	3	Richmond	+11:00	M.C.G.	2017-03-23 19:20:00	Richmond	100	5	132	

{'input': 'How many games have Christian Petracca and Clayton Oliver played together?',
 'output': 'Christian Petracca and Clayton Oliver have played 157 games together.'}

In [12]:
user_question = "What are the top 5 games by disposals but 0 brownlow votes? Who, when, and how many disposals?"
agent_executor.invoke(user_question)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'game, player_stats'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete INTEGER, 
	hbehinds INTEGER, 
	ascore INTEGER, 
	roundname TEXT, 
	year INTEGER, 
	is_final INTEGER, 
	hscore INTEGER, 
	abehinds INTEGER, 
	is_grand_final INTEGER, 
	updated TEXT, 
	hgoals INTEGER, 
	timestr TEXT, 
	hteam TEXT, 
	winnerteamid REAL, 
	ateamid INTEGER, 
	PRIMARY KEY (game_id)
)

/*
10 rows from game table:
round	agoals	game_id	unixtime	date	hteamid	ateam	tz	venue	localtime	winner	complete	hbehinds	ascore	roundname	year	is_final	hscore	abehinds	is_grand_final	updated	hgoals	timestr	hteam	winnerteamid	ateamid
1	20	1	1490257200	2017-03-23 19:20:00	3	Richmond	+11:00	M.C.G.	2017-03-23 19:20:00	Richm

{'input': 'What are the top 5 games by disposals but 0 brownlow votes? Who, when, and how many disposals?',
 'output': 'The top 5 games by disposals but with 0 Brownlow votes are as follows:\n\n1. Player: Matt Crouch, Disposals: 47\n2. Player: Tom Mitchell, Disposals: 46\n3. Player: Lachie Neale, Disposals: 46\n4. Player: Matt Priddis, Disposals: 45\n5. Player: Matthew Boyd, Disposals: 45'}

In [13]:
user_question = "How many times has a player kicked 9 goals and at least 1 behind in a game? Tell me about those games, player, year, round etc."
agent_executor.invoke(user_question)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


game, player_stats
Invoking: `sql_db_schema` with `{'table_names': 'game'}`



CREATE TABLE game (
	round INTEGER, 
	agoals INTEGER, 
	game_id INTEGER, 
	unixtime INTEGER, 
	date TEXT, 
	hteamid INTEGER, 
	ateam TEXT, 
	tz TEXT, 
	venue TEXT, 
	localtime TEXT, 
	winner TEXT, 
	complete INTEGER, 
	hbehinds INTEGER, 
	ascore INTEGER, 
	roundname TEXT, 
	year INTEGER, 
	is_final INTEGER, 
	hscore INTEGER, 
	abehinds INTEGER, 
	is_grand_final INTEGER, 
	updated TEXT, 
	hgoals INTEGER, 
	timestr TEXT, 
	hteam TEXT, 
	winnerteamid REAL, 
	ateamid INTEGER, 
	PRIMARY KEY (game_id)
)

/*
10 rows from game table:
round	agoals	game_id	unixtime	date	hteamid	ateam	tz	venue	localtime	winner	complete	hbehinds	ascore	roundname	year	is_final	hscore	abehinds	is_grand_final	updated	hgoals	timestr	hteam	winnerteamid	ateamid
1	20	1	1490257200	2017-03-23 19:20:00	3	Richmond	+11:00	M.C.G.	2017-03-23 19:20:00	Richmond	100	5	132	

{'input': 'How many times has a player kicked 9 goals and at least 1 behind in a game? Tell me about those games, player, year, round etc.',
 'output': 'Players who have kicked 9 goals and at least 1 behind in a game are as follows:\n\n1. **Gary Ablett**\n   - Round: 9\n   - Year: 1996\n   - Goals: 10\n   - Behinds: 8\n\n2. **Stewart Loewe**\n   - Round: 13\n   - Year: 1996\n   - Goals: 14\n   - Behinds: 12\n\n3. **Tony Lockett**\n   - Round: 18\n   - Year: 1998\n   - Goals: 15\n   - Behinds: 12\n\n4. **Jason Dunstall**\n   - Round: 11\n   - Year: 1996\n   - Goals: 16\n   - Behinds: 12\n\n5. **Jason Heatley**\n   - Round: 14\n   - Year: 1997\n   - Goals: 20\n   - Behinds: 13\n\nand so on...'}

In [14]:
print(" 'output': 'Players who have kicked 9 goals and at least 1 behind in a game are as follows:\n\n1. **Gary Ablett**\n   - Round: 9\n   - Year: 1996\n   - Goals: 10\n   - Behinds: 8\n\n2. **Stewart Loewe**\n   - Round: 13\n   - Year: 1996\n   - Goals: 14\n   - Behinds: 12\n\n3. **Tony Lockett**\n   - Round: 18\n   - Year: 1998\n   - Goals: 15\n   - Behinds: 12\n\n4. **Jason Dunstall**\n   - Round: 11\n   - Year: 1996\n   - Goals: 16\n   - Behinds: 12\n\n5. **Jason Heatley**\n   - Round: 14\n   - Year: 1997\n   - Goals: 20\n   - Behinds: 13\n\nand so on...'}")

 'output': 'Players who have kicked 9 goals and at least 1 behind in a game are as follows:

1. **Gary Ablett**
   - Round: 9
   - Year: 1996
   - Goals: 10
   - Behinds: 8

2. **Stewart Loewe**
   - Round: 13
   - Year: 1996
   - Goals: 14
   - Behinds: 12

3. **Tony Lockett**
   - Round: 18
   - Year: 1998
   - Goals: 15
   - Behinds: 12

4. **Jason Dunstall**
   - Round: 11
   - Year: 1996
   - Goals: 16
   - Behinds: 12

5. **Jason Heatley**
   - Round: 14
   - Year: 1997
   - Goals: 20
   - Behinds: 13

and so on...'}


In [20]:
user_question = "How many goals did Zak Butters kick last year?"

db_chain.run(TEMPLATE.format(user_question=user_question))




> Entering new SQLDatabaseChain chain...
You are Anthony (Huddo) Hudson, an AFL commentator and stat expert. You also know SQL and have access to an SQLite3 database of AFL statistics.
Given an input question, first create a syntactically correct SQLite3 query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

player_stats and game.

Few shot examples:
Question: "How many goals did Zak Butters kick in 2023?"
SQLQuery: "SELECT SUM(goals) FROM player_stats WHERE player_name = 'Zak Butters' AND year = 2023"

Question: "How many goals did Max Gawn kick last year?"
SQLQuery = "SELECT goals FROM player_stats WHERE player_name = 'Max Gawn' AND year = strftime('%Y', date('now', '-1 year'))"

Whenever you are asked a question, please return additional information that might be implied by the questio

'Zak Butters kicked 10 goals last year.'

In [6]:
user_question = "How good is Richmond?"
res = db_chain.run(TEMPLATE.format(user_question=user_question))



> Entering new SQLDatabaseChain chain...
You are Anthony (Huddo) Hudson, an AFL commentator and stat expert. You also know SQL and have access to an SQLite3 database of AFL statistics.
Given an input question, first create a syntactically correct SQLite3 query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

player_stats and game.

Few shot examples:
Question: "How many goals did Zak Butters kick in 2023?"
SQLQuery: "SELECT SUM(goals) FROM player_stats WHERE player_name = 'Zak Butters' AND year = 2023"

Question: "How many goals did Max Gawn kick last year?"
SQLQuery = "SELECT goals FROM player_stats WHERE player_name = 'Max Gawn' AND year = strftime('%Y', date('now', '-1 year'))"

Whenever you are asked a question, please return additional information that might be implied by the questio